In [91]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [92]:
#Read Data from soruces

# Read the sensor feature file (590 columns, no header)
df_features = pd.read_csv("C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom.data",
                          delim_whitespace=True,  # or sep=' '
                          header=None)

# Read the labels file (two columns: label and timestamp)
df_labels = pd.read_csv("C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom_labels.data",
                        delim_whitespace=True,
                        header=None)

# df_features.head()
# df_labels.head()

C:\Users\Prajakta B\AppData\Local\Temp\ipykernel_14520\885378504.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_features = pd.read_csv("C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom.data",
C:\Users\Prajakta B\AppData\Local\Temp\ipykernel_14520\885378504.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_labels = pd.read_csv("C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom_labels.data",


In [93]:
#Combining data and creating proper structure of table


# Loading dataset 
try:
    secom_data = pd.read_csv('C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom.data', delim_whitespace=True, header=None)
    labels = pd.read_csv('C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom_labels.data', delim_whitespace=True, header=None)
except FileNotFoundError as e:
    print(f"Error: {e}. Please verify file paths and names.")
    exit()
    
# Rename columns like feature_1,2.. target_1,2..
num_sensors = secom_data.shape[1]  # Number of sensor columns
new_columns = [f'feature_{i+1}' for i in range(num_sensors)] 
secom_data.columns = new_columns

labels.columns = ["is_faulty", "date_time"]

df = pd.concat([
          secom_data, labels
], axis=1)

C:\Users\Prajakta B\AppData\Local\Temp\ipykernel_14520\4161458270.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  secom_data = pd.read_csv('C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom.data', delim_whitespace=True, header=None)
C:\Users\Prajakta B\AppData\Local\Temp\ipykernel_14520\4161458270.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  labels = pd.read_csv('C:/Users/Prajakta B/Desktop/SECOM_DATA/secom/secom_labels.data', delim_whitespace=True, header=None)


In [94]:
# Test & train dataset split with stratified sampling
from sklearn.model_selection import train_test_split

feature_colms = [col for col in df.columns if col not in ['is_faulty']]
# Features: all columns starting with 'feature_'
X = df[feature_colms]

# Target: 'is_faulty'
y = df['is_faulty']

# Show frequency distribution before sampling
print("Frequency distribution BEFORE sampling:")
print(y.value_counts(normalize=True).rename('proportion').to_frame())
print(y.value_counts().rename('count').to_frame())
print('-' * 40)

# Stratified train-test split (67% train, 33% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.33,        # 33% test, 67% train
    random_state=42,       # seed value
    stratify=y             # Stratified split based on is_faulty
)


# Show frequency distribution in train set
print("Frequency distribution in TRAIN set:")
print(y_train.value_counts(normalize=True).rename('proportion').to_frame())
print(y_train.value_counts().rename('count').to_frame())
print('-' * 40)

# Show frequency distribution in test set
print("Frequency distribution in TEST set:")
print(y_test.value_counts(normalize=True).rename('proportion').to_frame())
print(y_test.value_counts().rename('count').to_frame())
print('-' * 40)

train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

train_df_clean = pd.concat([X_train, y_train], axis=1)
# Display datasets
# train_df
# test_df 
# train_df_clean

Frequency distribution BEFORE sampling:
           proportion
is_faulty            
-1           0.933631
 1           0.066369
           count
is_faulty       
-1          1463
 1           104
----------------------------------------
Frequency distribution in TRAIN set:
           proportion
is_faulty            
-1            0.93327
 1            0.06673
           count
is_faulty       
-1           979
 1            70
----------------------------------------
Frequency distribution in TEST set:
           proportion
is_faulty            
-1           0.934363
 1           0.065637
           count
is_faulty       
-1           484
 1            34
----------------------------------------


In [95]:
# Data cleaning in train dataset train_df_clean
# zero volatality columns

# Count initial columns (including target)
initial_col_count = train_df_clean.shape[1]
print(f"\nInitial number of columns: {initial_col_count}")

# Identify feature columns (exclude target)
feature_cols = [col for col in df.columns if col not in ['is_faulty', 'date_time']]

# Find zero-volatility columns (std = 0)
stds = train_df_clean[feature_cols].std()
zero_vol_cols = stds[stds == 0].index.tolist()

# Remove zero-volatility columns
train_df_clean = train_df_clean.drop(columns=zero_vol_cols)

# Count final columns (including target)
final_col_count = train_df_clean.shape[1]
print(f"Number of zero-volatility columns removed: {len(zero_vol_cols)}")
print(f"Final number of columns: {final_col_count}")
# print(train_df_clean)



Initial number of columns: 592
Number of zero-volatility columns removed: 116
Final number of columns: 476


In [96]:
# missing values column dropping
# Count columns before removal
before_cols = train_df_clean.shape[1]
print(f"Number of columns before removal: {before_cols}")

# Calculate the percentage of missing values in each column
missing_percent = train_df_clean.isnull().mean()

# Identify columns with more than 50% missing values
cols_to_drop = missing_percent[missing_percent > 0.50].index.tolist()

print(f"Number of columns with >50% missing values: {len(cols_to_drop)}")
print("Columns to drop due to missing values >50%:")
print(cols_to_drop)

# Drop these columns
train_df_clean = train_df_clean.drop(columns=cols_to_drop)

# Count columns after removal
after_cols = train_df_clean.shape[1]
print(f"Number of columns after removal: {after_cols}")
print(f"Number of columns removed: {before_cols - after_cols}")


Number of columns before removal: 476
Number of columns with >50% missing values: 24
Columns to drop due to missing values >50%:
['feature_86', 'feature_110', 'feature_111', 'feature_112', 'feature_158', 'feature_159', 'feature_221', 'feature_245', 'feature_246', 'feature_247', 'feature_293', 'feature_294', 'feature_359', 'feature_383', 'feature_384', 'feature_385', 'feature_493', 'feature_517', 'feature_518', 'feature_519', 'feature_579', 'feature_580', 'feature_581', 'feature_582']
Number of columns after removal: 452
Number of columns removed: 24


In [97]:
# # Handle missing values with KNN
from sklearn.impute import KNNImputer

# Identify feature columns (exclude target)
feature_cols = [col for col in train_df_clean.columns if col not in ['is_faulty', 'date_time']]

# Select only numeric columns among features
numeric_feature_cols = train_df_clean[feature_cols].select_dtypes(include=['float64', 'int64']).columns

# Store 'is_faulty' and 'date_time' separately
meta_cols = train_df_clean[['is_faulty', 'date_time']].copy()

# Apply KNN imputer only to numeric feature columns
imputer = KNNImputer(n_neighbors=3)
imputed_features = pd.DataFrame(
    imputer.fit_transform(train_df_clean[numeric_feature_cols]),
    columns=numeric_feature_cols,
    index=train_df_clean.index
)

# If you have any non-numeric feature columns, add them back (if needed)
non_numeric_cols = [col for col in feature_cols if col not in numeric_feature_cols]
imputed_df = pd.concat([imputed_features, train_df_clean[non_numeric_cols]], axis=1)

# Append 'is_faulty' and 'date_time' columns back
train_df_clean = pd.concat([imputed_df, meta_cols], axis=1)

print(train_df_clean)


      feature_1  feature_2  feature_3  feature_4  feature_5  feature_7  \
1263    2983.73    2459.87  2162.1333   998.9095     0.8826   104.9722   
999     3045.11    2444.87  2215.1778  2192.1867     1.8829    85.6589   
203     2981.95    2279.48  2205.2889  1630.3112     1.2733    98.8056   
396     3127.07    2478.97  2198.7222  1534.2053     0.9374   104.1989   
193     3024.15    2521.24  2191.7889  1133.3967     1.3808   106.6711   
...         ...        ...        ...        ...        ...        ...   
57      2935.94    2586.05  2164.4111  1206.6031     0.9799   100.5189   
365     2988.92    2460.91  2178.0778   941.9524     0.8039   104.0167   
1420    2975.74    2517.35  2162.5556  1041.0369     1.4305   100.4111   
113     2928.16    2523.21  2210.6111  1184.6481     1.2577   102.9356   
240     2900.69    2483.06  2191.1111  1564.6023     0.9366   102.5100   

      feature_8  feature_9  feature_10  feature_11  ...  feature_583  \
1263     0.1246     1.3897     -0.0091 

In [98]:
# High Correlation columns
# Initial column count
# initial_cols = train_df_clean.shape[1]

# #  Identify feature columns (exclude target and date)
# feature_cols = [col for col in train_df_clean.columns if col not in ['is_faulty', 'date_time']]

# # Compute correlation matrix and find highly correlated columns to drop
# corr_matrix = train_df_clean[feature_cols].corr().abs()
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
# threshold = 0.8
# high_corr_drop = [col for col in upper.columns if any(upper[col] > threshold)]
# # train_df_clean = train_df_clean.drop(columns=high_corr_drop)

# # Print results
# print(f"Columns dropped due to high correlation: {len(high_corr_drop)}")
# print("\nList of dropped columns:")
# print(high_corr_drop)

# # Final count
# final_cols = train_df_clean.shape[1]
# print(f"\nInitial columns: {initial_cols}")
# print(f"Final columns: {final_cols}")

from sklearn.feature_selection import mutual_info_classif
import numpy as np
import pandas as pd

# Initial column count
initial_cols = train_df_clean.shape[1]

# Identify feature columns (exclude target and date)
feature_cols = [col for col in train_df_clean.columns if col not in ['is_faulty', 'date_time']]

# Ensure all features are numeric 
X = train_df_clean[feature_cols]
if X.select_dtypes(include=['object', 'category']).shape[1] > 0:
    X = pd.get_dummies(X)
X = X.fillna(0)

# Compute MI scores
y = train_df_clean['is_faulty'].values
mi_scores = mutual_info_classif(X, y)
mi_series = pd.Series(mi_scores, index=X.columns)

# Compute correlation matrix and its upper triangle
corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

threshold = 0.8
to_drop = set()

# For each pair of highly correlated columns, drop the one with lower MI
for col in upper.columns:
    for row in upper.index:
        if pd.notnull(upper.loc[row, col]) and upper.loc[row, col] > threshold:
            if mi_series[col] < mi_series[row]:
                to_drop.add(col)
            else:
                to_drop.add(row)

# Drop the selected columns
train_df_clean = train_df_clean.drop(columns=list(to_drop))

# Print results
print(f"Columns dropped due to high correlation: {len(to_drop)}")
print("\nList of dropped columns:")
print(list(to_drop))

# Final count
final_cols = train_df_clean.shape[1]
print(f"\nInitial columns: {initial_cols}")
print(f"Final columns: {final_cols}")






Columns dropped due to high correlation: 232

List of dropped columns:
['feature_575', 'feature_357', 'feature_387', 'feature_239', 'feature_411', 'feature_287', 'feature_295', 'feature_197', 'feature_211', 'feature_557', 'feature_319', 'feature_432', 'feature_449', 'feature_148', 'feature_5', 'feature_561', 'feature_204', 'feature_542', 'feature_141', 'feature_143', 'feature_428', 'feature_209', 'feature_340', 'feature_205', 'feature_284', 'feature_139', 'feature_160', 'feature_174', 'feature_308', 'feature_206', 'feature_368', 'feature_389', 'feature_479', 'feature_144', 'feature_164', 'feature_437', 'feature_312', 'feature_453', 'feature_470', 'feature_18', 'feature_118', 'feature_54', 'feature_250', 'feature_229', 'feature_447', 'feature_172', 'feature_348', 'feature_456', 'feature_251', 'feature_168', 'feature_74', 'feature_351', 'feature_360', 'feature_472', 'feature_407', 'feature_37', 'feature_280', 'feature_249', 'feature_307', 'feature_311', 'feature_276', 'feature_125', 'fea

In [101]:
import numpy as np

# Select numeric columns, excluding target and date if needed
numeric_cols = train_df_clean.select_dtypes(include=[np.number]).columns
numeric_cols = [col for col in numeric_cols if col not in ['is_faulty', 'date_time']]

outlier_columns = []

for col in numeric_cols:
    data = train_df_clean[col]
    mean = data.mean()
    std = data.std()
    lower = mean - 3 * std
    upper = mean + 3 * std
    outliers = (data < lower) | (data > upper)
    if outliers.any():
        outlier_columns.append(col)

print("Columns containing outliers (3-sigma rule):")
print(outlier_columns)
print("count:")
print(len(outlier_columns))



Columns containing outliers (3-sigma rule):
['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_15', 'feature_16', 'feature_19', 'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_30', 'feature_33', 'feature_34', 'feature_36', 'feature_38', 'feature_39', 'feature_40', 'feature_42', 'feature_44', 'feature_45', 'feature_49', 'feature_51', 'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_62', 'feature_63', 'feature_64', 'feature_66', 'feature_69', 'feature_71', 'feature_72', 'feature_73', 'feature_75', 'feature_76', 'feature_77', 'feature_78', 'feature_79', 'feature_80', 'feature_82', 'feature_83', 'feature_84', 'feature_85', 'feature_87', 'feature_88', 'feature_89', 'feature_90', 'feature_91', 'feature_92', 'feature_93', 'feature_95', 'feature_96', 'feature_101', 'feature_102', 'feature_103', 'featur